In [71]:
import geopandas as gpd
import pandas as pd
import yaml
import xarray as xr

import os

In [58]:
with open('../../config/satvis.yaml', 'r') as f:
    config = yaml.safe_load(f)

dir_dict = config['data_directories']
sub_basin_shp = dir_dict['sub_basin_shp']
cyano_data_shp = dir_dict['cyano_data_shp']

print(sub_basin_shp)
print(cyano_data_shp)

C:\Arbetsmapp\Shapefiler\Sub-basins_Baltic_Sea\Havsomr_SVAR_2016_3b.shp
C:\Arbetsmapp\BAWS\shp_data


# BAWS data

In [59]:
# TODO Convert to GeoPackage and add both shape and raster data to GeoDataFrame

In [67]:
cyano_data_file = gpd.read_file('data/cyano_daymap_20220815.shp')
cyano_data_file

,class,geometry
0,1,"POLYGON ((869261.000 7354123.000, 873261.000 7..."
1,1,"POLYGON ((1114261.000 7354123.000, 1123261.000..."
2,1,"POLYGON ((1090261.000 7353123.000, 1092261.000..."
3,1,"POLYGON ((1098261.000 7351123.000, 1100261.000..."
4,1,"POLYGON ((232261.000 7354123.000, 236261.000 7..."
...,...,...
7757,1,"POLYGON ((500261.000 5959123.000, 502261.000 5..."
7758,1,"POLYGON ((553261.000 5993123.000, 555261.000 5..."
7759,1,"POLYGON ((642261.000 5961123.000, 645261.000 5..."
7760,1,"POLYGON ((753261.000 5981123.000, 756261.000 5..."


In [73]:
file_ts = pd.Timestamp(os.path.basename('data/cyano_daymap_20220815.shp').split('.')[0].split('_')[-1])
file_ts

Timestamp('2022-08-15 00:00:00')

In [75]:
# TODO Change to create a time multiindex
cyano_data_file['time'] = file_ts
cyano_data_file

,class,geometry,time
0,1,"POLYGON ((869261.000 7354123.000, 873261.000 7...",2022-08-15
1,1,"POLYGON ((1114261.000 7354123.000, 1123261.000...",2022-08-15
2,1,"POLYGON ((1090261.000 7353123.000, 1092261.000...",2022-08-15
3,1,"POLYGON ((1098261.000 7351123.000, 1100261.000...",2022-08-15
4,1,"POLYGON ((232261.000 7354123.000, 236261.000 7...",2022-08-15
...,...,...,...
7757,1,"POLYGON ((500261.000 5959123.000, 502261.000 5...",2022-08-15
7758,1,"POLYGON ((553261.000 5993123.000, 555261.000 5...",2022-08-15
7759,1,"POLYGON ((642261.000 5961123.000, 645261.000 5...",2022-08-15
7760,1,"POLYGON ((753261.000 5981123.000, 756261.000 5...",2022-08-15


In [60]:
def generate_filepaths(directory, pattern='', not_pattern='DUMMY_PATTERN',
                       pattern_list=[], endswith='',
                       only_from_dir=True):
    for path, subdir, fids in os.walk(directory):
        if only_from_dir:
            if path != directory:
                continue
        # Generator function (uses yield) https://docs.python.org/3/glossary.html#term-generator
        for f in fids:
            if pattern in f and not_pattern not in f and f.endswith(endswith):
                if any(pattern_list):
                    for pat in pattern_list:
                        if pat in f:
                            yield os.path.abspath(os.path.join(path, f))
                else:
                    yield os.path.abspath(os.path.join(path, f))

In [63]:
year = 2022
data_files = generate_filepaths(cyano_data_shp, pattern=year, endswith='.shp')
geodataframes = []

for cyano_data_file in data_files:
    gdf = gpd.read_file(cyano_data_file)
    # filename = os.path.basename(cyano_data_file)
    file_ts = pd.Timestamp(os.path.basename(cyano_data_file).split('.')[0].split('_')[-1])
    date = pd.to_datetime(file_ts)
    gdf = xr.Dataset.from_dataframe(gdf)
    ds = ds.assign_coords(time=date)
    # ds['geometry'] = xr.DataArray(geometry, dims=['index'])
    # Tried but doesn't work
    # ds = ds.assign_attrs(crs='epsg:3006', description='Dataset projection')
    datasets.append(ds)
ds_all = xr.concat(datasets, dim='time')

TypeError: expected string or bytes-like object, got 'generator'

# Basin data
Creates the GeoPackage file 'basins.gpkg' containing SVAR BASIN_NR numeration, geometry and basin_name for the sub-basins of the Baltic Sea according to Havsomr_SVAR_2016_3b

In [37]:
basin_mapping_SVAR = {
    1: 'Bottenviken',
    2: 'Norra Kvarken',
    3: 'Bottenhavet',
    4: 'Ålands hav',
    5: 'Skärgårdshavet',
    6: 'Finska viken',
    7: 'Norra Gotlandshavet',
    8: 'Västra Gotlandshavet',
    9: 'Östra Gotlandshavet',
    10: 'Rigabukten',
    11: 'Gdanskbukten',
    12: 'Bornholmshavet och Hanöbukten',
    13: 'Arkonahavet och Södra Öresund',
    14: 'Bälthavet',
    15: 'Öresund',
    16: 'Kattegatt',
    17: 'Skagerrak',
}

In [47]:
gdf_basin = gpd.read_file(sub_basin_shp)
basin_geometries = gdf_basin[['BASIN_NR', 'geometry']]
basin_data = basin_geometries.dissolve(by='BASIN_NR', as_index=True)
basin_data['basin_name'] = [basin_mapping_SVAR[item] for item in basin_data.index]

In [48]:
basin_data

,geometry,basin_name
BASIN_NR,,
1,"POLYGON ((788351.558 7113779.095, 788331.895 7...",Bottenviken
2,"POLYGON ((743322.192 7034002.034, 735055.884 7...",Norra Kvarken
3,"POLYGON ((620922.743 6729219.402, 620757.133 6...",Bottenhavet
4,"POLYGON ((709486.835 6669336.804, 709413.026 6...",Ålands hav
5,"POLYGON ((941642.960 6656019.222, 941484.624 6...",Skärgårdshavet
6,"POLYGON ((984634.061 6596697.898, 984670.443 6...",Finska viken
7,"POLYGON ((693957.780 6565996.762, 693875.447 6...",Norra Gotlandshavet
8,"POLYGON ((549687.035 6208329.209, 549711.191 6...",Västra Gotlandshavet
9,"POLYGON ((706381.131 6322005.715, 706379.902 6...",Östra Gotlandshavet


In [53]:
basin_data.to_file('basins.gpkg')

# Aggregate data

In [54]:
basins = gpd.read_file('basins.gpkg')

In [55]:
basins.crs

<Projected CRS: EPSG:3006>
Name: SWEREF99 TM
Axis Info [cartesian]:
- N[north]: Northing (metre)
- E[east]: Easting (metre)
Area of Use:
- name: Sweden - onshore and offshore.
- bounds: (10.03, 54.96, 24.17, 69.07)
Coordinate Operation:
- name: SWEREF99 TM
- method: Transverse Mercator
Datum: SWEREF99
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

# Tests

In [ ]:
daygdf = gpd.read_file('data/aggregation_2022.shp')

In [ ]:
daygdf

37

In [ ]:
total_area = daygdf.area.sum() / 1000000.
total_area

245094.0

In [ ]:
area_days = []

In [ ]:
for n in daygdf['class'].unique():
    boolean = daygdf['class'] == n
    area_days.append(daygdf.loc[boolean, :].area.sum() / 1000000. * n)

In [ ]:
boolean = daygdf['class'] == 3
daygdf.loc[boolean, :].area.sum() / 1000000 * 3.

64089.0

In [ ]:
daygdf['class'].unique()

array([ 2,  1,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       19, 18, 20, 21, 22, 23, 25, 24, 26, 27, 28, 29, 30, 31, 33, 32, 35,
       34, 36, 37], dtype=int64)

In [ ]:
boolean = daygdf['class'] == 2

In [ ]:
 area_days

[44884.0,
 26332.0,
 64089.0,
 72188.0,
 75740.0,
 81672.0,
 92106.0,
 103728.0,
 115893.0,
 122950.0,
 123827.0,
 118776.0,
 113100.0,
 107310.0,
 105735.0,
 101840.0,
 92531.0,
 62624.0,
 80946.0,
 51880.0,
 41160.0,
 34210.0,
 29348.0,
 26800.0,
 28056.0,
 24570.0,
 21303.0,
 15344.0,
 9280.0,
 5700.0,
 4495.0,
 858.0,
 2144.0,
 385.0,
 340.0,
 180.0,
 74.0]

In [ ]:
daygdf.loc[boolean, :].area.sum()

30872000000.0

In [ ]:
sum(area_days) / float(total_area)

1.0734599205706847

$\displaystyle a_i$ is equal to the extent that is covered by surface accumulations of cyanobacteria during ($\displaystyle i$) number of days

$\displaystyle A = \frac{\sum a_i \times i}{\sum i} =$ Normalized extent of cyanobacteria bloom

$\displaystyle T = \frac{\sum a_i \times i}{\sum a_i} =$ Normalized duration of cyanobacteria bloom

$\displaystyle I = A \times T =$ Normalized intensity of cyanobacteria bloom